In [33]:
import pandas as pd

# read xlsx file
df = pd.read_excel("/Users/johri/Projects/AIEarthHack/AI EarthHack Dataset.xlsx")
print(df.columns)

# df = pd.read_csv("AI EarthHack Dataset.xlsx")

# # print columns
# print(df.columns)

# filter entries with less than 10 words in the solution column
df = df[df['solution'].str.split().str.len().gt(20)]

# filter entries with similar solution 
df = df.drop_duplicates(subset=['solution'])
print(df.shape)



Index(['id', 'problem', 'solution'], dtype='object')
(1208, 3)


In [42]:
pip install --upgrade openai


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [43]:
import json

from openai import OpenAI

client = OpenAI(
    api_key="API-KEY",
)

def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    ret_json = json.dumps(response.choices[0].message.content.strip())
    
    return ret_json


# def chat_gpt(prompt):
#     response = openai.ChatCompletion.create(
#         model="gpt-3.5-turbo",
#         messages=[{"role": "user", "content": prompt}]
#     )
#     ret_json = json.dumps(response.choices[0].message.content.strip())
    
#     return ret_json

top3 = df.head(3)
top3

,id,problem,solution
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m..."
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single..."
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa..."


In [125]:
import json

min_word_count = 10
min_bullet = 2
max_bullet = 5
key_features = "objective, relevance_to_circular_economy, feasibility, impact, technical_complexity, innovation, sustainability, market_potential, competitive_landscape, cost-effectiveness"
dict_keys = "ratings_objective, ratings_relevance_to_circular_economy, ratings_feasibility, ratings_impact, ratings_technical_complexity, ratings_innovation, ratings_sustainability, ratings_market_potential, ratings_competitive_landscape, ratings_cost-effectiveness, explanation_objective, explanation_relevance_to_circular_economy, explanation_feasibility, explanation_impact, explanation_technical_complexity, explanation_innovation, explanation_sustainability, explanation_market_potential, explanation_competitive_landscape, explanation_cost-effectiveness"
df_final = pd.DataFrame(columns=['id', 'problem', 'solution', 'ratings_objective', 'ratings_relevance_to_circular_economy', 'ratings_feasibility', 'ratings_impact', 'ratings_technical_complexity', 'ratings_innovation', 'ratings_sustainability', 'ratings_market_potential', 'ratings_competitive_landscape', 'ratings_cost-effectiveness', 'explanation_objective', 'explanation_relevance_to_circular_economy', 'explanation_feasibility', 'explanation_impact', 'explanation_technical_complexity', 'explanation_innovation', 'explanation_sustainability', 'explanation_market_potential', 'explanation_competitive_landscape', 'explanation_cost-effectiveness'])


for inx, row in top3.iterrows():
    problem_statement = row['problem'].strip()
    solution_statement = row['solution'].strip()

    prompt = f"""Extract the main startegy from the text and give the pipline architecture of the following proposed approach in short bullet points. Each bullet point does not exceed {min_word_count} words. In total limit to {min_bullet}-{max_bullet} bullet points. The problem statement is: "{problem_statement}". The solution is "{solution_statement}".Identify key points: {key_features}. Also rate these features out of 100 based on it's overall effectiveness. If any feature can not be identified mention it as "NA". Output the result should be in json format following the below format of keys: {dict_keys}. """

    response_json = chat_gpt(prompt)

    # Removing extra escape characters and outer quotes
    cleaned_json = response_json.replace('\\n', "").replace("\\","")

    print(cleaned_json)
    cleaned_json = cleaned_json.strip('"\'').replace("\'","")
    print(cleaned_json)

    # Convert the cleaned JSON-like string to a JSON object
    json_object = json.loads(cleaned_json)

    # Merge the updated dictionaries
    json_object['id'] = row['id']
    json_object['problem'] = problem_statement
    json_object['solution'] = solution_statement
    print(json_object)

    json_df = pd.DataFrame(json_object, index=[1])   
    df_final = pd.concat([df_final, json_df], ignore_index=True)

df_final

"{    "ratings_objective": 80,    "ratings_relevance_to_circular_economy": 90,    "ratings_feasibility": 85,    "ratings_impact": 90,    "ratings_technical_complexity": 60,    "ratings_innovation": 85,    "ratings_sustainability": 95,    "ratings_market_potential": 95,    "ratings_competitive_landscape": 80,    "ratings_cost-effectiveness": 90,    "explanation_objective": "Modular Construction as a solution to construction waste",    "explanation_relevance_to_circular_economy": "Promotes recycling and reuse, aligning with circular economy principles",    "explanation_feasibility": "Proven feasibility, being implemented in global markets",    "explanation_impact": "Significantly reduces waste and conserves resources",    "explanation_technical_complexity": "Requires engineering expertise for component design and assembly",    "explanation_innovation": "Innovative approach compared to traditional construction methods",    "explanation_sustainability": "Transition from linear 'take, make 

,id,problem,solution,ratings_objective,ratings_relevance_to_circular_economy,ratings_feasibility,ratings_impact,ratings_technical_complexity,ratings_innovation,ratings_sustainability,...,explanation_objective,explanation_relevance_to_circular_economy,explanation_feasibility,explanation_impact,explanation_technical_complexity,explanation_innovation,explanation_sustainability,explanation_market_potential,explanation_competitive_landscape,explanation_cost-effectiveness
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",80,90,85,90,60,85,95,...,Modular Construction as a solution to construc...,"Promotes recycling and reuse, aligning with ci...","Proven feasibility, being implemented in globa...",Significantly reduces waste and conserves reso...,Requires engineering expertise for component d...,Innovative approach compared to traditional co...,"Transition from linear take, make and dispose ...",Scalable solution with high potential for real...,Competitive advantage in terms of waste reduct...,Substantial financial savings due to decreased...
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",90,95,85,100,80,95,90,...,The objective is to harness wind energy withou...,The proposed approach aligns with the circular...,The approach utilizes existing technology and ...,The approach has a high impact as it addresses...,The technical complexity is moderate as it req...,The approach introduces a revolutionary method...,The approach promotes sustainable energy gener...,The market potential is high as there is a gro...,The competitive landscape is moderately compet...,The approach is cost-effective compared to tra...
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",80,90,70,85,60,75,85,...,Implement a book swap program in educational i...,Reduces the need for new book production and p...,Feasible to create a platform for book exchang...,Significantly lowers resource depletion and wa...,Moderate complexity in developing the digital ...,Integrating gamification leads to cultural shi...,Contributes to circular economy by reducing wa...,Has the potential to be adopted widely in educ...,Competition may exist from existing book renta...,"Saves money for students, parents, and schools..."


In [126]:
df_final

,id,problem,solution,ratings_objective,ratings_relevance_to_circular_economy,ratings_feasibility,ratings_impact,ratings_technical_complexity,ratings_innovation,ratings_sustainability,...,explanation_objective,explanation_relevance_to_circular_economy,explanation_feasibility,explanation_impact,explanation_technical_complexity,explanation_innovation,explanation_sustainability,explanation_market_potential,explanation_competitive_landscape,explanation_cost-effectiveness
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",80,90,85,90,60,85,95,...,Modular Construction as a solution to construc...,"Promotes recycling and reuse, aligning with ci...","Proven feasibility, being implemented in globa...",Significantly reduces waste and conserves reso...,Requires engineering expertise for component d...,Innovative approach compared to traditional co...,"Transition from linear take, make and dispose ...",Scalable solution with high potential for real...,Competitive advantage in terms of waste reduct...,Substantial financial savings due to decreased...
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",90,95,85,100,80,95,90,...,The objective is to harness wind energy withou...,The proposed approach aligns with the circular...,The approach utilizes existing technology and ...,The approach has a high impact as it addresses...,The technical complexity is moderate as it req...,The approach introduces a revolutionary method...,The approach promotes sustainable energy gener...,The market potential is high as there is a gro...,The competitive landscape is moderately compet...,The approach is cost-effective compared to tra...
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",80,90,70,85,60,75,85,...,Implement a book swap program in educational i...,Reduces the need for new book production and p...,Feasible to create a platform for book exchang...,Significantly lowers resource depletion and wa...,Moderate complexity in developing the digital ...,Integrating gamification leads to cultural shi...,Contributes to circular economy by reducing wa...,Has the potential to be adopted widely in educ...,Competition may exist from existing book renta...,"Saves money for students, parents, and schools..."
